# A notebook to create convoluted bulk mRNA-seq samples for processing

In [11]:
import os
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipynb
import subprocess
from glob import glob

from ipynb.fs.full.python_cmd_tools import *

# load sample meta data

In [2]:
# Env variables
envs = [line.strip().split() for line in open('./env_paths')]
envs = {i[0]: i[1] for i in envs}

In [3]:
# generated by NextFlow in fist step
sample_sheet = join(envs['MAQC_DATA'], 'samplesheet/samplesheet.csv')
sample_sheet = pd.read_csv(sample_sheet)

In [4]:
# select the .fastq files that are bulk
bulk_samples = sample_sheet[sample_sheet.experiment_title.str.contains('bulk')]
bulk_samples = bulk_samples.reset_index(drop=True)

In [5]:
bulk_samples[['sample_title', 'experiment_title', 'run_alias', 'library_source', 'read_count', 'sample_description', 'description']]#.loc[:, 'experiment_title'][1]

,sample_title,experiment_title,run_alias,library_source,read_count,sample_description,description
0,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395BL_1_S10_R1_001.fastq.gz,TRANSCRIPTOMIC,26549469,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...
1,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395BL_2_S11_R1_001.fastq.gz,TRANSCRIPTOMIC,28768620,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...
2,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395BL_3_S12_R1_001.fastq.gz,TRANSCRIPTOMIC,29246313,HCC1395 BL,NextSeq 550 sequencing; bulk RNA-seq of HCC139...
3,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395_2_S14_R1_001.fastq.gz,TRANSCRIPTOMIC,28095730,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...
4,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395_3_S15_R1_001.fastq.gz,TRANSCRIPTOMIC,27883943,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...
5,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...,HCC1395_1_S13_R1_001.fastq.gz,TRANSCRIPTOMIC,30200567,HCC1395,NextSeq 550 sequencing; bulk RNA-seq of HCC139...


# prepare the convoluted samples for testing

Each cell line has a single paired end library which has been sequenced three times.

I will create three samples from the first replicate of HCC1395 with 10%, 50%, and 90% contamination from HCC1395 BL

In [6]:
f_cancer = bulk_samples.loc[5].fastq_1.replace('/run/media/ian', '/home/jovyan')
r_cancer = bulk_samples.loc[5].fastq_2.replace('/run/media/ian', '/home/jovyan')

f_bcell = bulk_samples.loc[0].fastq_1.replace('/run/media/ian', '/home/jovyan')
r_bcell = bulk_samples.loc[0].fastq_2.replace('/run/media/ian', '/home/jovyan')

In [7]:
! mkdir -p ./output

In [8]:
%%time

conv_portions = [.10, .50, .90]
file_pairs = [[f_cancer, f_bcell, 1], [r_cancer, r_bcell, 2]]
out_path = './output'
for cp in conv_portions:
    for fp in file_pairs:
        file_name = str(cp * 100).replace('.0', 'Percent')
        fA = fp[0]
        fB = fp[1]
        direction = str(fp[2])
        output_fastq = join(out_path, 'conv_'+file_name+'_'+direction+'.fastq')
        convolute_raw_data(fA, fB, output_fastq, cp)

CPU times: user 23.6 ms, sys: 169 ms, total: 193 ms
Wall time: 14min 1s


Check that new files have the correct number of reads

In [21]:
%%time

new_files = glob(out_path+'/*')
for p in [10, 50, 90]:
    temp_list = [i for i in new_files if str(p) in i]
    tf0 = temp_list[0]
    tf1 = temp_list[1]
    assert count_fastq_reads(tf0) == count_fastq_reads(tf1)

CPU times: user 19.7 ms, sys: 49.4 ms, total: 69.1 ms
Wall time: 1min 18s
